# OpenChat Round Table

In [ ]:
%cd /content/drive/MyDrive/NLP/semevaltask9

/content/drive/MyDrive/NLP/semevaltask9


In [ ]:
import torch
import numpy as np
import pandas as pd
import random
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score

In [ ]:
GLOBAL_SEED = 255

np.random.seed(GLOBAL_SEED)
random.seed(GLOBAL_SEED)
torch.manual_seed(GLOBAL_SEED)
torch.use_deterministic_algorithms(True)
%env CUBLAS_WORKSPACE_CONFIG=:4096:8

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


env: CUBLAS_WORKSPACE_CONFIG=:4096:8


## Use Gemini

In [ ]:
!pip install hugchat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.9 MB/s eta 0:00:00


In [ ]:
from hugchat import hugchat
from hugchat.login import Login

# Log in to huggingface and grant authorization to huggingchat
sign = Login(mail, password)
cookies = sign.login()

# Save cookies to the local directory
cookie_path_dir = "./cookies_snapshot"
sign.saveCookiesToDir(cookie_path_dir)

'./cookies_snapshot/baktash.ansari1381@gmail.com.json'

In [ ]:
chatbot = hugchat.ChatBot(cookies=cookies.get_dict())

In [ ]:
# Create a new conversation with mixtral
chatbot.switch_llm(5)
id = chatbot.new_conversation()
chatbot.change_conversation(id)

### Load datasets

In [ ]:
# Import Dataset
data = np.load("./Data/SP_train.npy", allow_pickle = True ).tolist()
SP_train = pd.DataFrame(data)
data = np.load("./Data/SP_test.npy", allow_pickle = True ).tolist()
SP_test = pd.DataFrame(data)
data = np.load("./Data/SP_test_answer.npy", allow_pickle = True ).tolist()
SP_test_answer = pd.DataFrame(data)

data = np.load("./Data/WP_train.npy", allow_pickle = True ).tolist()
WP_train = pd.DataFrame(data)
data = np.load("./Data/WP_test.npy", allow_pickle = True ).tolist()
WP_test = pd.DataFrame(data)
data = np.load("./Data/WP_test_answer.npy", allow_pickle = True ).tolist()
WP_test_answer = pd.DataFrame(data)

data = np.load("./Data/sentence_puzzle.npy", allow_pickle = True ).tolist()
SP = pd.DataFrame(data)
data = np.load("./Data/word_puzzle.npy", allow_pickle = True ).tolist()
WP = pd.DataFrame(data)

In [ ]:
for index, row in SP_test.iterrows():
  SP_test.at[index, 'label'] = int(SP_test_answer.loc[index][1])

In [ ]:
for index, row in WP_test.iterrows():
  WP_test.at[index, 'label'] = int(WP_test_answer.loc[index][1])

## Initial Round

### Create Prompt

In [ ]:
prompt = """Please answer the question with step-by-step reasoning. Also,evaluate your confidence level (between 0.0 and 1.0) to indicate the possibility of your answer being right. Please give the answer in this template:
Solution:
Reasoning:
Confidence:

Question: """

In [ ]:
def generate_prompt(item, prompt):

  labels = ["A", "B", "C", "D"]
  prompt += f'{item["question"]}\n'
  for index, choice in enumerate(item["choice_list"]):
    prompt += f'{labels[index]} : {choice}\n'
  return prompt

In [ ]:
print(generate_prompt(SP_test.loc[115], prompt))

Please answer the question with step-by-step reasoning. Also,evaluate your confidence level (between 0.0 and 1.0) to indicate the possibility of your answer being right. Please give the answer in this template:
Solution:
Reasoning:
Confidence:

Question: A professional fisherman caught 30 fish during a 5-day tournament. Each day, he caught three more fish than the day before. How many fish did the fisherman catch on the first day?
A : Two.
B : One.
C : Three.
D : None of above.



### Run inference

In [ ]:
import time
# OpenResultsR0 = []
for index, row in SP_test.iterrows():
  if index + 1 < 28:
    continue
  chatbot.switch_llm(5)
  id = chatbot.new_conversation()
  chatbot.change_conversation(id)
  query_result = chatbot.query(generate_prompt(row, prompt))
  print(index + 1)
  print(query_result)
  OpenResultsR0.append({"id" : index+1, "response" : query_result.text})
  print()
  time.sleep(5)

28
 Solution: C
Reasoning: Since we know that the only living beings mentioned are Romeo and Juliet, and they are described as "fish," it's logical to deduce that they must be pet fish kept in an aquarium. Therefore, when the shelf was disturbed by the passing train, the aquarium could have been knocked over, causing their tanks to break and leading to their deaths. This eliminates options A, B, and D.
Confidence: 0.95

29
 Solution: B
Reasoning: In most sports teams, it is usually the responsibility of the team captain to lead the team when the vice-coach or head coach cannot be present for a game. They are elected by their fellow players and have experience working closely with the coaching staff. Therefore, they understand the strategies and tactics required to guide the team effectively during games. Hence, option B is the correct answer.
Confidence: 1.0

30
 Solution: B
Reasoning: Since the question states that the person screamed "Not the gun" before the incident occurred, we can

In [ ]:
OpenResultsR0

[{'id': 1,
  'response': ' Solution: A\nReasoning: Since it\'s mentioned that the diligent farmer is the son of the lazy farmer, we can infer that the lazy farmer must be either the father or the mother of the diligent farmer. However, since it\'s also stated that the lazy farmer isn\'t the father of the diligent farmer, the only other option left is that the lazy farmer is the mother of the diligent farmer. This would make sense because parents can still love and care for their children even if they themselves aren\'t diligent workers. So, the correct answer is A: "The lazy farmer is his mother."\nConfidence: 1.0'},
 {'id': 2,
  'response': " Solution: B : The rumble of the train moved the shelf which crushed them.\nReasoning: Given the context, we know that there is a nearby railroad track and the only furniture present are a bookcase and a bed. It's unlikely that either option A or C would be correct because they don't involve any of the mentioned furniture. Option D can be eliminat

In [ ]:
# save dataframe:
OpenR0 = pd.DataFrame(OpenResultsR0)
OpenR0.to_csv("./OpenR0.csv")

In [ ]:
OpenR0

,id,response
0,1,Solution: A\nReasoning: Since it's mentioned ...
1,2,Solution: B : The rumble of the train moved t...
2,3,Solution: C : It happens every year.\nReasoni...
3,4,Solution: B : The captain.\nReasoning: In mos...
4,5,Solution: C : Five people.\nReasoning: We kno...
...,...,...
115,116,Solution: C : Three.\nReasoning: Let's denote...
116,117,Solution: C\nReasoning: Since the internet wa...
117,118,Solution: B : A preacher.\nReasoning: The sta...
118,119,"Solution: B : Six. The bed's four legs, plus ..."


In [ ]:
# create structural data frame:
Structural_Open_R0 = []

for index, item in enumerate(OpenResultsR0):
  print(index)
  print(item['response'].split('Solution: '))

  solution = item['response'].split('Solution: ')[1][0]
  reasoning = item['response'].split('Reasoning:')[1].split('Confidence: ')[0]
  confidence = item['response'].split('Confidence: ')[1].split('\n')[0].strip()

  Structural_Open_R0.append({"Solution" : solution, "Reasoning" : reasoning, "Confidence" : confidence})

Structural_Open_R0

0
[' ', 'A\nReasoning: Since it\'s mentioned that the diligent farmer is the son of the lazy farmer, we can infer that the lazy farmer must be either the father or the mother of the diligent farmer. However, since it\'s also stated that the lazy farmer isn\'t the father of the diligent farmer, the only other option left is that the lazy farmer is the mother of the diligent farmer. This would make sense because parents can still love and care for their children even if they themselves aren\'t diligent workers. So, the correct answer is A: "The lazy farmer is his mother."\nConfidence: 1.0']
1
[' ', "B : The rumble of the train moved the shelf which crushed them.\nReasoning: Given the context, we know that there is a nearby railroad track and the only furniture present are a bookcase and a bed. It's unlikely that either option A or C would be correct because they don't involve any of the mentioned furniture. Option D can be eliminated as well since one of the other options seems more plau

[{'Solution': 'A',
  'Reasoning': ' Since it\'s mentioned that the diligent farmer is the son of the lazy farmer, we can infer that the lazy farmer must be either the father or the mother of the diligent farmer. However, since it\'s also stated that the lazy farmer isn\'t the father of the diligent farmer, the only other option left is that the lazy farmer is the mother of the diligent farmer. This would make sense because parents can still love and care for their children even if they themselves aren\'t diligent workers. So, the correct answer is A: "The lazy farmer is his mother."\n',
  'Confidence': '1.0'},
 {'Solution': 'B',
  'Reasoning': " Given the context, we know that there is a nearby railroad track and the only furniture present are a bookcase and a bed. It's unlikely that either option A or C would be correct because they don't involve any of the mentioned furniture. Option D can be eliminated as well since one of the other options seems more plausible. Therefore, the most 

In [ ]:
finalDF = pd.DataFrame(Structural_Open_R0)

In [ ]:
finalDF

,Solution,Reasoning,Confidence
0,A,Since it's mentioned that the diligent farmer...,1.0
1,B,"Given the context, we know that there is a ne...",0.75
2,C,This problem is about understanding how birth...,1.0
3,B,"In most sports organizations, the hierarchy i...",1.0
4,C,We know that 5% of the population has an unli...,1.0
...,...,...,...
115,C,Let's denote x as the number of fish the fish...,1.0
116,C,"Since the internet was disconnected, Bob woul...",0.95
117,B,The statement refers to a person who marries ...,0.95
118,B,We need to count all the legs that are touchi...,1.0. This calculation accounts for every anima...


In [ ]:
finalDF.to_csv('./Openchat_R0.csv')

In [ ]:
mapping = {'A' : 0, 'B' : 1, 'C' : 2, 'D' : 3}

for index, item in enumerate(finalDF['Solution'].tolist()):
  print(mapping[item])

0
1
2
1
2
0
0
2
1
2
1
0
2
0
2
1
0
2
1
1
1
2
0
0
2
1
0
2
1
1
1
3
2
0
1
2
1
0
2
0
1
0
0
1
0
2
2
2
0
1
0
0
1
1
1
0
1
1
1
0
2
1
0
1
3
0
2
1
2
3
2
2
2
1
1
2
0
1
1
1
2
0
3
0
0
1
2
2
1
2
2
2
2
0
2
1
3
3
0
1
3
1
0
2
3
2
0
1
0
1
1
3
0
3
0
2
2
1
1
2
